# 01 - Data Gathering

## Description
This notebook gathers the necissary data from 3 different data sources:
 - 2022Q3 Divvy bike/trip data found on [Divvy's Data Portal](https://divvy-tripdata.s3.amazonaws.com/index.html)
 - Estimated travel time using  [Open Source Routing Machine (OSRM) API](http://project-osrm.org/docs/v5.10.0/api/#general-options)
 - Historic weather data from [OpenWeather API](https://openweathermap.org/api/one-call-3#data)

## Step 0: Setup Libraries and File Locations

### Import Libraries

In [29]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import requests
import zipfile
import io
import datetime
import json
from tqdm import tqdm
from scrapy.selector import Selector
import config

### Create file location

In [2]:
ROOT = os.getcwd()
SAVE_FILES = os.path.join(ROOT, "DATA")
SAVE_FILES

'C:\\Users\\Nicholas\\Desktop\\Masters - Classes\\MSDS436\\Final\\MSDS436-FINAL\\DATA'

## Step 1: Pull Divvy Bike Data

### Pull keys from website using BeautifulSoup

In [3]:
main_url = 'https://divvy-tripdata.s3.amazonaws.com'
page = requests.get(main_url)
soup = BeautifulSoup(page.content, 'html.parser')

# print(soup.prettify())

In [4]:
zip_keys = soup.findAll('key')
len(zip_keys)

50

### Loop through Keys and only keep divvy-tripdata

In [5]:
key_ls = []

for i in tqdm(range(len(zip_keys))):
    key_ls.append(zip_keys[i].text)

key_ls_clean = [ x for x in key_ls if "divvy-tripdata" in x ]

key_ls_clean[27:30]

100%|██████████████████████████████████████████████████| 50/50 [00:00<?, ?it/s]


['202207-divvy-tripdata.zip',
 '202208-divvy-tripdata.zip',
 '202209-divvy-tripdata.zip']

### Pull and save all files

In [ ]:
for zip_f in key_ls_clean[27:30]:
    r = requests.get(f"https://divvy-tripdata.s3.amazonaws.com/{zip_f}")
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(SAVE_FILES)

### For our data purpose, read in the 2022Q3 files and create 1 master file
- '202207-divvy-tripdata.csv',
- '202208-divvy-tripdata.csv',
- '202209-divvy-tripdata.csv'

In [6]:
# Get 202207, 202208, and 202209 files and save file path in list
file_ls = []

for file in os.listdir(SAVE_FILES):
    if file.endswith(".csv"):
        file_ls.append(os.path.join(SAVE_FILES, file))
    
file_ls

['C:\\Users\\Nicholas\\Desktop\\Masters - Classes\\MSDS436\\Final\\MSDS436-FINAL\\DATA\\202207-divvy-tripdata.csv',
 'C:\\Users\\Nicholas\\Desktop\\Masters - Classes\\MSDS436\\Final\\MSDS436-FINAL\\DATA\\202208-divvy-tripdata.csv',
 'C:\\Users\\Nicholas\\Desktop\\Masters - Classes\\MSDS436\\Final\\MSDS436-FINAL\\DATA\\202209-divvy-publictripdata.csv']

In [7]:
# Loop through files, create pandas data frame, and save in dictonary
df_dict = {}

for i in file_ls:
    for num in range(len(file_ls)):
        df = pd.read_csv(i)
        df_dict[f"df_{num}"] = df

In [8]:
# Grab dictonary keys and check dataframe
dict_keys_ls = list(df_dict.keys())
df_dict[dict_keys_ls[2]].head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5156990AC19CA285,electric_bike,2022-09-01 08:36:22,2022-09-01 08:39:05,NaN,NaN,California Ave & Milwaukee Ave,13084,41.93,-87.69,41.922695,-87.697153,casual
1,E12D4A16BF51C274,electric_bike,2022-09-01 17:11:29,2022-09-01 17:14:45,NaN,NaN,NaN,NaN,41.87,-87.62,41.870000,-87.620000,casual
2,A02B53CD7DB72DD7,electric_bike,2022-09-01 17:15:50,2022-09-01 17:16:12,NaN,NaN,NaN,NaN,41.87,-87.62,41.870000,-87.620000,casual
3,C82E05FEE872DF11,electric_bike,2022-09-01 09:00:28,2022-09-01 09:10:32,NaN,NaN,NaN,NaN,41.93,-87.69,41.940000,-87.670000,casual
4,4DEEB4550A266AE1,electric_bike,2022-09-01 07:30:11,2022-09-01 07:32:36,NaN,NaN,NaN,NaN,41.92,-87.73,41.920000,-87.730000,casual


In [24]:
sep_df = df_dict[dict_keys_ls[2]]
sep_df.index.name = 'row'
len(sep_df)

701339

In [ ]:
# # Concat all 3 data frames and generate Q3_df
# Q3_df = pd.concat([df_dict[dict_keys_ls[0]], df_dict[dict_keys_ls[1]], df_dict[dict_keys_ls[2]]], ignore_index=True, axis=0)
# display(len(Q3_df))
# display(Q3_df.head())

## Step 2: Get estimated travel time

In [11]:
#new method per https://github.com/Project-OSRM/osrm-backend/issues/6258
def get_distance_bike(point1: dict, point2: dict) -> tuple:
    """Gets distance between two points en route using http://project-osrm.org/docs/v5.10.0/api/#nearest-service"""
    
    url = f"""https://routing.openstreetmap.de/routed-bike/route/v1/biking/{point1["start_lng"]},{point1["start_lat"]};{point2["end_lng"]},{point2["end_lat"]}?overview=false&alternatives=false"""
    r = requests.get(url)
    
    # get the distance from the returned values
    route = json.loads(r.content)["routes"][0]
    return (route["distance"], route["duration"])

In [13]:
# get the distances and durations
dist_array_bike = []
for i , r in tqdm(sep_df.iterrows()):
    try:
        point1 = {"start_lat": r["start_lat"], "start_lng": r["start_lng"]}
        point2 = {"end_lat": r["end_lat"], "end_lng": r["end_lng"]}
        dist, duration = get_distance_bike(point1, point2)
        #dist = geodesic((i_lat, i_lon), (o["CapitalLatitude"], o["CapitalLongitude"])).km
        dist_array_bike.append((i, duration, dist))
    except KeyError:
        dist_array_bike.append((i, 0, 0))
        continue

69853it [8:00:29,  2.42it/s]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [15]:
# ensure values are different
# print(dist_array_car) ---> [(0, 800.9, 3224.7), (1, 1289.7, 4141.1)]
print(len(dist_array_bike))

69853


In [22]:
distances_bike_df = pd.DataFrame(dist_array_bike,columns=["row","duration (s)","distance (m)"])
distances_bike_df.head()

,row,duration (s),distance (m)
0,0,493.6,1871.5
1,1,0.0,0.0
2,2,0.0,0.0
3,3,704.8,2821.1
4,4,0.0,0.0


In [26]:
# Matches key value for 'row' or any other unique identifier we want to assign later on
sep_dis_df = pd.merge(sep_df.iloc[:69854], distances_bike_df, on='row', how='right').drop('row', axis=1)
sep_dis_df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration (s),distance (m)
0,5156990AC19CA285,electric_bike,2022-09-01 08:36:22,2022-09-01 08:39:05,NaN,NaN,California Ave & Milwaukee Ave,13084,41.930000,-87.690000,41.922695,-87.697153,casual,493.6,1871.5
1,E12D4A16BF51C274,electric_bike,2022-09-01 17:11:29,2022-09-01 17:14:45,NaN,NaN,NaN,NaN,41.870000,-87.620000,41.870000,-87.620000,casual,0.0,0.0
2,A02B53CD7DB72DD7,electric_bike,2022-09-01 17:15:50,2022-09-01 17:16:12,NaN,NaN,NaN,NaN,41.870000,-87.620000,41.870000,-87.620000,casual,0.0,0.0
3,C82E05FEE872DF11,electric_bike,2022-09-01 09:00:28,2022-09-01 09:10:32,NaN,NaN,NaN,NaN,41.930000,-87.690000,41.940000,-87.670000,casual,704.8,2821.1
4,4DEEB4550A266AE1,electric_bike,2022-09-01 07:30:11,2022-09-01 07:32:36,NaN,NaN,NaN,NaN,41.920000,-87.730000,41.920000,-87.730000,casual,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69848,CFCC2E803DB5E54C,electric_bike,2022-09-14 17:41:13,2022-09-14 18:00:59,Michigan Ave & Washington St,13001,Noble St & Milwaukee Ave,13290,41.883859,-87.624371,41.900680,-87.662600,member,1113.9,4195.6
69849,0EE32B5D67A40E6C,classic_bike,2022-09-06 17:02:21,2022-09-06 17:10:11,Orleans St & Merchandise Mart Plaza,TA1305000022,Orleans St & Chestnut St (NEXT Apts),620,41.888243,-87.636390,41.898203,-87.637536,member,339.3,1200.6
69850,A193953F3BAF66EE,classic_bike,2022-09-08 18:53:54,2022-09-08 19:06:18,Wells St & Elm St,KA1504000135,Desplaines St & Kinzie St,TA1306000003,41.903222,-87.634324,41.888716,-87.644448,member,658.6,2487.1
69851,0E37389776C4B4A3,electric_bike,2022-09-28 17:12:21,2022-09-28 17:25:37,Clark St & Schiller St,TA1309000024,Desplaines St & Kinzie St,TA1306000003,41.907966,-87.631688,41.888716,-87.644448,member,833.1,3229.2


In [27]:
# Save data frame
sep_dis_df.to_csv("202209_divvy_distance.csv")

In [ ]:
# export master file
# Q3_df.to_csv("2022Q3_divvy-tripdata.csv")

## Collect historic weather data
**NOTE:** the config.py file contains api_key

In [31]:
# round lat and long to 2 decimal places (needed for API)
sep_dis_df['start_lat_clean'] = sep_dis_df['start_lat'].round(2)
sep_dis_df['start_lng_clean'] = sep_dis_df['start_lng'].round(2)

sep_dis_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration (s),distance (m),start_lat_clean,start_lng_clean
0,5156990AC19CA285,electric_bike,2022-09-01 08:36:22,2022-09-01 08:39:05,NaN,NaN,California Ave & Milwaukee Ave,13084,41.93,-87.69,41.922695,-87.697153,casual,493.6,1871.5,41.93,-87.69
1,E12D4A16BF51C274,electric_bike,2022-09-01 17:11:29,2022-09-01 17:14:45,NaN,NaN,NaN,NaN,41.87,-87.62,41.870000,-87.620000,casual,0.0,0.0,41.87,-87.62
2,A02B53CD7DB72DD7,electric_bike,2022-09-01 17:15:50,2022-09-01 17:16:12,NaN,NaN,NaN,NaN,41.87,-87.62,41.870000,-87.620000,casual,0.0,0.0,41.87,-87.62
3,C82E05FEE872DF11,electric_bike,2022-09-01 09:00:28,2022-09-01 09:10:32,NaN,NaN,NaN,NaN,41.93,-87.69,41.940000,-87.670000,casual,704.8,2821.1,41.93,-87.69
4,4DEEB4550A266AE1,electric_bike,2022-09-01 07:30:11,2022-09-01 07:32:36,NaN,NaN,NaN,NaN,41.92,-87.73,41.920000,-87.730000,casual,0.0,0.0,41.92,-87.73


In [41]:
sep_dis_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69853 entries, 0 to 69852
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ride_id             69853 non-null  object        
 1   rideable_type       69853 non-null  object        
 2   started_at          69853 non-null  object        
 3   ended_at            69853 non-null  object        
 4   start_station_name  30559 non-null  object        
 5   start_station_id    30559 non-null  object        
 6   end_station_name    30516 non-null  object        
 7   end_station_id      30516 non-null  object        
 8   start_lat           69853 non-null  float64       
 9   start_lng           69853 non-null  float64       
 10  end_lat             69842 non-null  float64       
 11  end_lng             69842 non-null  float64       
 12  member_casual       69853 non-null  object        
 13  duration (s)        69853 non-null  float64   

In [44]:
sep_dis_df['unix_dt'] = pd.to_datetime(sep_dis_df['started_at'])
sep_dis_df['unix_dt'] = pd.to_datetime(sep_dis_df['unix_dt'])
sep_dis_df['unix_dt'] = pd.to_numeric(sep_dis_df['unix_dt'])
sep_dis_df['unix_dt'] = sep_dis_df['unix_dt'] // 10 ** 9

sep_dis_df.head()

In [46]:
def weathermap(latnum, lngnum, dtnum):
    '''
    Pulls weather data using lat, long, and unix_dt
    '''
    api_url = f"http://api.openweathermap.org/data/3.0/onecall/timemachine?lat={latnum}&lon={lngnum}&dt={dtnum}&units=imperial&appid={config.api_key}"
    response = requests.get(api_url)
    resp = response.json()
    
    temp = resp['data'][0]['temp']
    hum = resp['data'][0]['humidity']
    windsp = resp['data'][0]['wind_speed']
    weather = resp['data'][0]['weather'][0]['main']
    try:
        rain = resp['data'][0]['rain']['1h']
    except KeyError as ke:
        rain = 0    
    try:
        snow = resp['data'][0]['snow']['1h']
    except KeyError as ke:
        snow = 0
    
    return temp, hum, windsp, weather, rain, snow

In [51]:
temp_ls = []
hum_ls = []
windsp_ls = []
weather_ls = []
rain_ls = []
snow_ls = []


for i in tqdm(range(len(sep_dis_df))):
    try:
        latnum = sep_dis_df['start_lat'].iloc[i]
        lngnum = sep_dis_df['start_lng'].iloc[i]
        dtnum = sep_dis_df['unix_dt'].iloc[i]
        temp_ls.append(weathermap(latnum, lngnum, dtnum)[0])
        hum_ls.append(weathermap(latnum, lngnum, dtnum)[1])
        windsp_ls.append(weathermap(latnum, lngnum, dtnum)[2])
        weather_ls.append(weathermap(latnum, lngnum, dtnum)[3])
        rain_ls.append(weathermap(latnum, lngnum, dtnum)[4])
        snow_ls.append(weathermap(latnum, lngnum, dtnum)[5])
    except NameError:
        temp_ls.append('Nan')
        hum_ls.append('Nan')
        windsp_ls.append('Nan')
        weather_ls.append('Nan')
        rain_ls.append('Nan')
        snow_ls.append('Nan')
    
sep_dis_df['temp'] = temp_ls
sep_dis_df['hum'] = hum_ls
sep_dis_df['windsp'] = windsp_ls
sep_dis_df['weather'] = weather_ls
sep_dis_df['rain'] = rain_ls
sep_dis_df['snow'] = snow_ls

  0%|                                                | 0/69853 [00:00<?, ?it/s]


KeyError: 'data'

### Save data in AWS S3 Bucket

In [53]:
latnum = sep_dis_df['start_lat'].iloc[0]
latnum

41.93

In [54]:
lngnum = sep_dis_df['start_lng'].iloc[0]
lngnum

-87.69

In [55]:
dtnum = sep_dis_df['unix_dt'].iloc[i]
dtnum

1662021382

In [59]:
api_url = f"http://api.openweathermap.org/data/3.0/onecall/timemachine?lat={latnum}&lon={lngnum}&dt={dtnum}&units=imperial&appid={config.api_key}"
response = requests.get(api_url)
resp = response.json()

In [60]:
resp

{'cod': 429,
 'message': 'Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription https://openweathermap.org/price'}